### prompts using ChatPromptTemplate

In [218]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant.",),
    MessagesPlaceholder(variable_name="conversation"),
    HumanMessage(content="What's the capital of France?"),
])

history = [
    HumanMessage(content="Hi!"),
    AIMessage(content="Hello! How can I help?")
]

chat_prompt.format_messages(conversation=history)




[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I help?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the capital of France?", additional_kwargs={}, response_metadata={})]

### prompts using PromptTemplate and FewShotPromptTemplate

In [219]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
# from langchain.llms import OpenAI

examples = [
    {"input": "Hello", "output": "Bonjour"},
    {"input": "Goodbye", "output": "Au revoir"}
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="{input} -> {output}",
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="{input} ->",
    input_variables=["input"]
)


prompt.pretty_print()

print("--------------------------------")
example_prompt

Hello -> Bonjour

Goodbye -> Au revoir

{input} ->
--------------------------------


PromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, template='{input} -> {output}')

## Tested PydanticOutputParser - (Working)

In [208]:
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, JsonOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import (
    AIMessage, HumanMessage, SystemMessage
)

In [209]:
max_essay_length = 500

In [210]:
class ModelResponse(BaseModel):
    """Model response schema."""
    model_name: str = Field(description="Name of the model", max_length=20)
    response: str = Field(description="Response from the model, based on the question", max_length=max_essay_length)
    tagline: str = Field(description="response created by Vivek", max_length=50)

In [211]:
resp_parser = PydanticOutputParser(pydantic_object=ModelResponse)
resp_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Model response schema.", "properties": {"model_name": {"description": "Name of the model", "maxLength": 20, "title": "Model Name", "type": "string"}, "response": {"description": "Response from the model, based on the question", "maxLength": 500, "title": "Response", "type": "string"}, "tagline": {"description": "response created by Vivek", "maxLength": 50, "title": "Tagline", "type": "string"}}, "required": ["model_name", "response", "tagline"]}\n```'

In [212]:
model = init_chat_model(model="gemini-2.0-flash", model_provider="google_genai")

In [213]:
messages = PromptTemplate(template="""You are a helpful assistant.Reqspond to the user's query: {query}. 
                        you Must follow these :{format_instruction}\n 
                        IMPORTANT: The max length allowed for the essay is {max_essay_length} CHARACTERS (not words). 
                        Keep your response concise and under this limit""",
                        input_variables=["query"],
                        partial_variables={"format_instruction": resp_parser.get_format_instructions()}
                        )

In [214]:
messages.pretty_print()

You are a helpful assistant.Reqspond to the user's query: {query}. 
                        you Must follow these :The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Model response schema.", "properties": {"model_name": {"description": "Name of the model", "maxLength": 20, "title": "Model Name", "type": "string"}, "response": {"description": "Response from the model, based on the question", "maxLength": 500, "title": "Response", "type": "string"}, "tagline": {"description": "response created by Vivek", "maxLength": 50, "title": "Tagline", "type": "string"}}, "required": ["mode

In [215]:
prompt = messages.format(query="write an essay on misuse of smartphones", max_essay_length=max_essay_length)
prompt

'You are a helpful assistant.Reqspond to the user\'s query: write an essay on misuse of smartphones. \n                        you Must follow these :The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Model response schema.", "properties": {"model_name": {"description": "Name of the model", "maxLength": 20, "title": "Model Name", "type": "string"}, "response": {"description": "Response from the model, based on the question", "maxLength": 500, "title": "Response", "type": "string"}, "tagline": {"description": "response created by Vivek", "maxLength": 50, "title": "Taglin

In [216]:
# chain = model | resp_parser
# res = chain.invoke(prompt)
model_response = model.invoke(prompt)
print(model_response.content)

final_response = resp_parser.parse(model_response.content)


```json
{"model_name": "EssayWriter", "response": "Smartphones, while useful, are often misused. Overuse leads to addiction, impacting mental health & productivity. Excessive screen time strains eyes and disrupts sleep. Social isolation and cyberbullying are also concerns. Prioritizing real-life interactions is crucial.", "tagline": "Created by Vivek for responsible tech use."}
```


In [217]:
final_response.model_dump_json()
response_dict = final_response.model_dump()
response_dict

{'model_name': 'EssayWriter',
 'response': 'Smartphones, while useful, are often misused. Overuse leads to addiction, impacting mental health & productivity. Excessive screen time strains eyes and disrupts sleep. Social isolation and cyberbullying are also concerns. Prioritizing real-life interactions is crucial.',
 'tagline': 'Created by Vivek for responsible tech use.'}